Installing necessary Libraries

In [5]:
!pip install gputil

In [4]:
pip install --upgrade pip

In [6]:
pip install tensorflow


INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/276.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/276.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/276.5 MB 1.9 MB/s eta 0:02:29
   ---------------------------------------- 0.8/276.5 MB 1.7 MB/s eta 0:02:45
   ---------------------------------------- 1.3/276.5 MB 1.8 MB/s eta 0:02:32
   ---------------------------------------- 1.6/276.5 MB 1.8 MB/s eta 0:02:31
   ---------------------------------------- 2.1/276.5 MB 1.9 MB/s eta 0:02:25
   ---------------------------------------- 2.4/276.5 MB 1.8 MB/s eta 0:02:32
   ---------------------------------------- 2.9/276.5 MB 1.9 MB/s eta 0:02:27
   ---------------------------------------- 3.1/276.5 MB 1.9 MB/s eta 0:02:27
    --------------------------------------- 3.9/276.5 MB 2.0 MB/s eta 0:02:19
    -----

In [7]:
pip install GPUtil

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install opencv-python


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB ? eta -:--:--
    --------------------------------------- 0.8/38.8 MB 1.5 MB/s eta 0:00:27
   - -------------------------------------- 1.3/38.8 MB 1.9 MB/s eta 0:00:21
   - -------------------------------------- 1.6/38.8 MB 1.8 MB/s eta 0:00:21
   -- ------------------------------------- 2.1/38.8 MB 1.8 MB/s eta 0:00:21
   -- ------------------------------------- 2.4/38.8 MB 1.9 MB/s eta 0:00:20
   -- ------------------------------------- 2.9/38.8 MB 2.0 MB/s eta 0:00:19
   --- ------------------------------------ 3.4/38.8 MB 2.0 MB/s eta 0:00:18
   ---- ----------------------------------- 3.9/38.8 MB 2.1 MB/s eta 0:00:17
   ---- ----------------------------------- 4.5/38.8 MB 2.1 MB/s eta 0:00:17
   ----- ---------------------------------- 5.0/38.8 MB 2.2 MB/s eta 0:00:16
   ----- ----------

In [10]:
pip install matplotlib

   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.5 MB 1.5 MB/s eta 0:00:05
   ---- ----------------------------------- 0.8/7.5 MB 1.8 MB/s eta 0:00:04
   ------ --------------------------------- 1.3/7.5 MB 1.7 MB/s eta 0:00:04
   --------- ------------------------------ 1.8/7.5 MB 1.8 MB/s eta 0:00:04
   ------------ --------------------------- 2.4/7.5 MB 1.9 MB/s eta 0:00:03
   ------------- -------------------------- 2.6/7.5 MB 2.0 MB/s eta 0:00:03
   ---------------- ----------------------- 3.1/7.5 MB 2.0 MB/s eta 0:00:03
   ------------------- -------------------- 3.7/7.5 MB 2.1 MB/s eta 0:00:02
   ----------------------- ---------------- 4.5/7.5 MB 2.2 MB/s eta 0:00:02
   --------------------------- ------------ 5.2/7.5 MB 2.4 MB/s eta 0:00:01
   --------------------------------- ------ 6.3/7.5 MB 2.6 MB/s eta 0:00:01
   -----------------------

importing necessary libraries

In [6]:
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import psutil
import GPUtil
from tensorflow.keras.models import load_model

In [4]:
conda activate OCR


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --user easyocr

Note: you may need to restart the kernel to use updated packages.


Installing EasyOCR model

In [1]:
pip show easyocr

Name: easyocr
Version: 1.7.2
Summary: End-to-End Multi-Lingual Optical Character Recognition (OCR) Solution
Home-page: https://github.com/jaidedai/easyocr
Author: Rakpong Kittinaradorn
Author-email: r.kittinaradorn@gmail.com
License: Apache License 2.0
Location: c:\users\anupam\appdata\roaming\python\python38\site-packages
Requires: ninja, numpy, opencv-python-headless, Pillow, pyclipper, python-bidi, PyYAML, scikit-image, scipy, Shapely, torch, torchvision
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [8]:
import easyocr

In [11]:

def process_video_with_easyocr(video_path, output_path, use_gpu=False):
    # Initialize EasyOCR reader
    reader = easyocr.Reader(['en'], gpu=use_gpu)
    
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_time = 0
    frame_times = []
    cpu_usages = []
    gpu_usages = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time()
        
        # Perform OCR on the frame
        results = reader.readtext(frame, detail=0)  # detail=0 returns only the detected text
        end_time = time.time()

        processing_time = end_time - start_time
        total_time += processing_time
        frame_times.append(processing_time)

        # Capture CPU and GPU usage
        cpu_usage = psutil.cpu_percent(interval=None)
        gpu_usage = None
        if use_gpu:
            gpus = GPUtil.getGPUs()
            gpu_usage = gpus[0].load * 100 if gpus else 0

        cpu_usages.append(cpu_usage)
        gpu_usages.append(gpu_usage if use_gpu else 0)

        # Annotate video with detected text and performance metrics
        fontScale = 1.5
        thickness = 3
        text_y_position = 30
        
        for i, detected_text in enumerate(results):
            cv2.putText(frame, f"Detected Text: {detected_text}", (10, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0, 255, 0), thickness)
            text_y_position += 40
        
        cv2.putText(frame, f"Processing Time: {processing_time:.2f}s", (10, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0, 255, 0), thickness)
        cv2.putText(frame, f"CPU Usage: {cpu_usage:.2f}%", (10, text_y_position + 40), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0, 255, 0), thickness)
        if use_gpu:
            cv2.putText(frame, f"GPU Usage: {gpu_usage:.2f}%", (10, text_y_position + 80), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0, 255, 0), thickness)

        out.write(frame)
        frame_count += 1

    average_fps = frame_count / total_time
    print(f"Average FPS ({'GPU' if use_gpu else 'CPU'}): {average_fps:.2f}")

    cap.release()
    out.release()
    # cv2.destroyAllWindows()

    return average_fps, frame_times, cpu_usages, gpu_usages

# Video paths
video_path = 'input.mp4'
output_path_gpu = 'output_video_gpu.avi'
output_path_cpu = 'output_video_cpu.avi'

# Run processing on GPU and CPU
fps_gpu, frame_times_gpu, cpu_usages_gpu, gpu_usages = process_video_with_easyocr(video_path, output_path_gpu, use_gpu=True)
fps_cpu, frame_times_cpu, cpu_usages_cpu, _ = process_video_with_easyocr(video_path, output_path_cpu, use_gpu=False)

print(f"Average FPS (GPU): {fps_gpu:.2f}")
print(f"Average FPS (CPU): {fps_cpu:.2f}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.


Average FPS (GPU): 0.41
Average FPS (CPU): 0.41
Average FPS (GPU): 0.41
Average FPS (CPU): 0.41


In [10]:
video_path = '/content/input.mp4'
output_path_gpu = 'output_video_gpu.avi'
output_path_cpu = 'output_video_cpu.avi'

In [13]:
import numpy as np
import matplotlib.pyplot as plt

def plot_performance_comparison(frame_times_gpu, accuracies_gpu, frame_times_cpu, accuracies_cpu, cpu_usages_gpu, cpu_usages_cpu, fps_gpu, fps_cpu):
    plt.figure(figsize=(18, 10))

    # Subplot for Processing Time Comparison
    plt.subplot(2, 3, 1)
    plt.plot(frame_times_gpu, label='GPU Processing Time', color='blue')
    plt.plot(frame_times_cpu, label='CPU Processing Time', color='red')
    plt.title('Processing Time Comparison')
    plt.xlabel('Frame')
    plt.ylabel('Time (s)')
    plt.legend()

    # Subplot for Accuracy Comparison
    plt.subplot(2, 3, 2)
    plt.plot(accuracies_gpu, label='GPU Accuracy', color='blue')
    plt.plot(accuracies_cpu, label='CPU Accuracy', color='red')
    plt.title('Accuracy Comparison') 
    plt.xlabel('Frame')
    plt.ylabel('Accuracy')
    plt.legend()

    # Subplot for CPU Usage Comparison
    plt.subplot(2, 3, 3)
    plt.plot(cpu_usages_gpu, label='GPU CPU Usage', color='blue')
    plt.plot(cpu_usages_cpu, label='CPU Usage', color='red')
    plt.title('CPU Usage Comparison')
    plt.xlabel('Frame')
    plt.ylabel('CPU Usage (%)')
    plt.legend()

    # Subplot for Average FPS Comparison
    plt.subplot(2, 3, 5)
    plt.bar(['GPU', 'CPU'], [fps_gpu, fps_cpu], color=['blue', 'red'])
    plt.title('Average FPS Comparison')
    plt.ylabel('FPS')

    plt.tight_layout()
    plt.show()

# Assuming these metrics are already computed from your processing code
fps_gpu = 30.0  # Example value for GPU FPS
fps_cpu = 25.0  # Example value for CPU FPS
frame_times_gpu = np.random.rand(100) * 0.1  # Example frame times for GPU
frame_times_cpu = np.random.rand(100) * 0.15  # Example frame times for CPU
accuracies_gpu = np.random.rand(100) * 0.95  # Example accuracies for GPU
accuracies_cpu = np.random.rand(100) * 0.90  # Example accuracies for CPU
cpu_usages_gpu = np.random.rand(100) * 100  # Example CPU usages for GPU mode
cpu_usages_cpu = np.random.rand(100) * 100  # Example CPU usages for CPU mode

# Call the plotting function
plot_performance_comparison(frame_times_gpu, accuracies_gpu, frame_times_cpu, accuracies_cpu, cpu_usages_gpu, cpu_usages_cpu, fps_gpu, fps_cpu)

# Performance Evaluation
print("Performance Evaluation:")
print(f"FPS (GPU): {fps_gpu:.2f}")
print(f"FPS (CPU): {fps_cpu:.2f}")
print(f"Average CPU Usage (GPU mode): {np.mean(cpu_usages_gpu):.2f}%")
print(f"Average CPU Usage (CPU mode): {np.mean(cpu_usages_cpu):.2f}%")


NameError: name 'accuracies_gpu' is not defined